# Carte des arrondissements

Test du chrolopleth plotly avec la gEographie des arrondissements de montrEal

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import json

MAP_FPATH = '../assets/maps/arrondissements_montreal.geojson'

with open(MAP_FPATH) as f:
    arrond = json.load(f)
    
print(arrond.keys())
print(arrond['type'])
print(arrond['crs'])
print(arrond['features'])

id = 'no_qr'
fig = go.Figure(
    go.Choroplethmap(
        geojson=arrond,
        featureidkey='properties.'+id,
        locations=[f['properties'][id] for f in arrond['features']],
        z=[f['properties'][id] for f in arrond['features']],
        hovertext=[f['properties']['nom_qr'] for f in arrond['features']],
        colorscale="Viridis",
        marker_opacity=0.5, marker_line_width=0))
fig.update_layout(map=dict(center=dict(lat=45.5517, lon=-73.7073), style="carto-positron", zoom=8.7))
fig.show()

# Districts Electoraux de MontrEal
Il manque tout ce qui n'est pas techniquement partie de la ville de montrEal, e.g. Westmount

In [ ]:
import plotly.graph_objects as go
import json

MAP_FPATH = '../assets/maps/districts_montreal.geojson'

with open(MAP_FPATH) as f:
    districts = json.load(f)
    
print(districts.keys())
print(districts['type'])
print(districts['name'])
print(len(districts['features']))
print(districts['features'][0]['geometry'])
print(districts['features'][0].keys())
print(districts['features'][0]['properties'].keys())

id = 'id'
fig = go.Figure(go.Choroplethmap(
    geojson=districts,
    featureidkey='properties.'+id,
    locations=[f['properties'][id] for f in districts['features']],
    z=[f['properties'][id] for f in districts['features']],
    hovertext=[f['properties']['nom'] for f in districts['features']],
    marker_opacity=0.5, marker_line_width=0))
fig.update_layout(
    map=dict(
        center=dict(lat=45.5517, lon=-73.7073), zoom=8.7))
fig.show()

# Superposition des arrondissements et circos

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import json

ARROND_FPATH = '../assets/maps/arrondissements_montreal.geojson'
CIRCO_FPATH = '../assets/maps/districts_montreal.geojson'


with open(ARROND_FPATH) as f:
    arrond = json.load(f)
with open(CIRCO_FPATH) as f:
    districts = json.load(f)    

# En bleu les arrondissements
id = 'no_qr'
fig = go.Figure(
    go.Choroplethmap(
        geojson=arrond,
        featureidkey='properties.'+id,
        locations=[f['properties'][id] for f in arrond['features']],
        z=[0]*len(arrond['features']),
        hovertext=[f['properties']['nom_qr'] for f in arrond['features']],
        marker_opacity=0.5, marker_line_width=5, marker_line_color='blue'))

# En rouge les circonscriptions
id = 'id'
fig = fig.add_trace(go.Choroplethmap(
    geojson=districts,
    featureidkey='properties.'+id,
    locations=[f['properties'][id] for f in districts['features']],
    z=[1]*len(districts['features']),
    hovertext=[f['properties']['nom'] for f in districts['features']],
    colorscale="Viridis",
    marker_opacity=0.5, marker_line_width=5, marker_line_color='red'))
fig.update_layout(
    showlegend=True,
    map=dict(
        center=dict(lat=45.5517, lon=-73.7073), zoom=8.7))
fig.show()

# Test des 125 districts Electoraux du QuEbec

In [ ]:
import plotly.graph_objects as go
import numpy as np
import json

MAP_FPATH = '../assets/maps/districts_QC.geojson'

with open(MAP_FPATH) as f:
    districts = json.load(f)
    
districts['features'] = districts['features']
for i in range(len(districts['features'])):
    districts['features'][i]['properties']['ID'] = i

print(districts.keys())
print(districts['type'])
print(districts['name'])
print(districts['crs'])
print(len(districts['features']))
print(districts['features'][0].keys())
for key in districts['features'][0]['properties'].keys():
    print('|\t', key, ':', districts['features'][0]['properties'][key])

id='ID'
fig = go.Figure(go.Choroplethmap(
    geojson=districts,
    featureidkey='properties.'+id,
    locations=[f['properties'][id] for f in districts['features']],
    z=[f['properties'][id] for f in districts['features']],
    hovertext=[f['properties']['NM_CEP'] for f in districts['features']],
    marker_opacity=0.5, marker_line_width=1))

fig.update_geos(
    projection=dict(
        type="conic conformal",
        parallels=[50, 46],
    ))
fig.update_layout(
    map=dict(center=dict(lat=54, lon=-68.5), zoom=3.7),
    width=600, height=800)

fig.show()

# Test du dataset de données démographiques par circo

Plus utilisé, le code a été refactor dans `preprocess.py`

In [ ]:
import plotly.graph_objects as go
from unidecode import unidecode
import pandas as pd
import json

MAP_125_FPATH = '../assets/maps/districts_QC.geojson'
DEMOGRAPHICS_FPATH = '../assets/data/donneesSocio2021.csv'

# Get the map
with open(MAP_125_FPATH) as f:
    districts = json.load(f)
print('Number of features :', len(districts['features']))

# Clean the names
for i in range(len(districts['features'])):
    districts['features'][i]['properties']['NM_CEP'] = unidecode(districts['features'][i]['properties']['NM_CEP'])
# Roughly sort. NOT RELIABLE for 1-to-1 matching
districts['features'].sort(key=lambda x: x['properties']['NM_CEP'].lower())
 
# Add unique IDs
for i in range(len(districts['features'])):
    districts['features'][i]['properties']['ID'] = i

# Display the keys
print('Property keys for the map :')
for key in districts['features'][0]['properties'].keys():
    print('|\t', key, ':', type(districts['features'][0]['properties'][key]), '; example :', districts['features'][0]['properties'][key])

# Get the demographics
df = pd.read_csv(DEMOGRAPHICS_FPATH, sep=';')
df.rename(columns={s:unidecode(s) for s in df.columns}, inplace=True)
df.rename(columns={'Circonscription/ DSE 2021':'Circonscription'}, inplace=True)
print('\nDataframe columns :')
#print(*zip(df.columns, df.dtypes), sep='\n')
print(*df.columns, sep='\n')
df.rename(columns={s: s.strip() for s in df.columns}, inplace=True)

# The first row is a recap for the entire province, we drop it and clean
df = df.drop(df[df['Circonscription']=='Province'].index).reset_index(drop=True)
df['Circonscription'] = df['Circonscription'].map(lambda s: unidecode(s))
df['Circonscription'] = df['Circonscription'].map(lambda s: s.strip())
print('Number of districts :', len(df))

# Sort in the same way as the map
df.sort_values(by='Circonscription', inplace=True, key=lambda x: x.str.lower())
df = df.reset_index(drop=True)

def get_map(demographics, variable, opacity=0.5):
    z = demographics[variable].values
    if demographics[variable].dtype == 'object':
        z = [float(s[:-1].replace(',', '.')) for s in z]
    print(z)
    fig = go.Figure(go.Choroplethmap(
        geojson=districts,
        featureidkey='properties.ID',
        locations=[f['properties']['ID'] for f in districts['features']],
        z=z,
        hovertext=[f['properties']['NM_CEP'] for f in districts['features']],
        marker_opacity=opacity, marker_line_width=0))
    fig.update_geos(
        projection=dict(
            type="conic conformal",
            parallels=[50, 46]))
    fig.update_layout(
        map=dict(center=dict(lat=54, lon=-68.5), zoom=3.65),
        width=600, height=800)
    return fig


In [ ]:
for i, col in enumerate(df.columns):
    print(col, '| example :', df.loc[0, col])

In [ ]:
print(*df.dtypes, sep='\n')
print(*df.columns[df.dtypes=='int'], sep='\n')

relevant_columns = [
    'Circonscrition',
    'Population totale selon les groupes d\'age',
    'Age moyen',
    'Age median',
]
    


In [ ]:
## Unit tests, Assert that the dataframes are correctly ordered etc
all_map_districts = [f['properties']['NM_CEP'] for f in districts['features']]
print(len(df))

# Test Double inclusion
for i in range(125):
    assert df.loc[i, 'Circonscription'] in all_map_districts, f'{df.loc[i, "Circonscription"]} not in the map'
    assert all_map_districts[i] in df['Circonscription'].values, f'{districts["features"][i]["properties"]["NM_CEP"]} not in the dataframe'

# Test full ordering
for a,b in zip(all_map_districts, df['Circonscription']):
    assert a == b, f'{a} not equal to {b}'

In [ ]:
var = "Population totale agee de 15 ans et plus dans les menages prives selon le plus haut certificat, diplome ou grade"
var = "Immigrants" 
#var = "Population totale selon les groupes d'age"
fig = get_map(df, var, opacity=0.3)
fig.update_layout(title=var)
fig.show()

# Test du dataset de vote

In [ ]:
from maps import *
from preprocess import *

map_data = get_map_data()
df = get_elections_data()

print(len(df))
print(*df.columns, sep='\n')

In [ ]:
# Candidats par circonscription
print(df.groupby('nomCirconscription').apply(lambda x: x.shape[0], include_groups=False))

# Candidats par parti
print(df.groupby('abreviationPartiPolitique').apply(lambda x: x.shape[0], include_groups=False))

# Vote par parti
print(df.groupby('abreviationPartiPolitique').apply(lambda x: x['nbVoteValide'].sum(), include_groups=False))

# Vote par circonscription
print(df.groupby('nomCirconscription').apply(lambda x: x['nbVoteValide'].sum(), include_groups=False))

In [ ]:
df_grouped = vote_summary_by_circo(df)

print(df_grouped.columns)
print(df_grouped.head(5))

fig = get_map(map_data, df_grouped, 'nbElecteurInscrit', opacity=0.3)
fig.show()


# Test du code de preprocess

In [ ]:

from maps import *
from preprocess import *

map_data = get_map_data()
df = get_demographics_data()

for a,b in zip([f['properties']['NM_CEP'] for f in map_data['features']], df['Circonscription']):
    assert a == b, f'{a} not equal to {b}'

fig = get_map(map_data, df, 'Population totale selon les groupes d\'age', opacity=0.3)
fig.show()

# Test subdivisions

Making sure that I wrote the names correctly

In [ ]:
from preprocess import *

df = get_demographics_data()
#print(*df['Circonscription'].tolist(), sep='\n')

circo_groups = {
    'Montréal': [
        'Acadie',
        'Anjou-Louis-Riel',
        'Bourassa-Sauve',
        'Camille-Laurin',
        'D\'Arcy-McGee',
        'Gouin',
        'Hochelaga-Maisonneuve',
        'Jacques-Cartier',
        'Jeanne-Mance-Viger',
        'LaFontaine',
        'Laurier-Dorion',
        'Marguerite-Bourgeoys',
        'Marquette',
        'Maurice-Richard',
        'Mercier',
        'Mont-Royal-Outremont',
        'Nelligan',
        'Notre-Dame-de-Grace',
        'Pointe-aux-Trembles',
        'Robert-Baldwin',
        'Rosemont',
        'Saint-Henri-Sainte-Anne',
        'Saint-Laurent',
        'Sainte-Marie-Saint-Jacques',
        'Verdun',
        'Viau',
        'Westmount-Saint-Louis'],
    'Québec': [
        'Charlesbourg',
        'Charlevoix-Cote-de-Beaupre',
        'Chauveau',
        'Jean-Lesage',
        'Jean-Talon',
        'La Peltrie',
        'Louis-Hebert',
        'Montmorency',
        'Portneuf',
        'Taschereau',
        'Vanier-Les Rivieres'], 
}

for group in circo_groups.keys():
    for circo in circo_groups[group]:
        assert circo in df['Circonscription'].values, f'{circo} not in the dataframe'

## Test formatting demographics

In [ ]:
from preprocess import *

df = get_demographics_data()

for i in range(len(df.columns)):
    if df.dtypes.iloc[i] == 'object' and df.columns[i] not in ['Circonscription']:
        col = df.columns[i]
        print(col)
        df.iloc[:,i] = df.iloc[:,i].str[:-1].map(lambda s: float(s.replace(',', '.').replace(' ', '')) if s else 0)

## Countries GeoJSON

In [ ]:
import plotly.graph_objects as go
import json

path = '../assets/maps/countries.geojson'
with open(path) as f:
        map_data = json.load(f)

print(*map_data['features'][0]['properties'].keys(), sep='\n')

fig = go.Figure(go.Choroplethmap(
        geojson=map_data,
        featureidkey='properties.name',
        locations=[f['properties']['name'] for f in map_data['features']],
        z=[0]*len(map_data['features']),
        marker_opacity=0.5, marker_line_width=1))
fig.update_geos(
    projection=dict(
        type="conic conformal",
        parallels=[50, 46]))

## Test affichage par pays

In [ ]:
from maps import *
from preprocess import *
from unidecode import unidecode
import numpy as np

map_data = get_countries_mapdata()
#print(*map_data['features'][0]['properties'].keys(), sep='\n')
print(len(map_data['features']))
print(*[f['properties']['name']+' ; '+f['properties']['continent'] for f in map_data['features']], sep='\n')


In [ ]:
df = get_immigration_data()

print(df['Arrondissement'])

In [ ]:

"""
Start idx for origin country/continent : 134
End idx : 192 included 
"""

indices = {
    'all': np.arange(134, 193),
    'Americas': {
        'main': np.arange(135, 145),
        'other': 145
    },
    'Europe': {
        'main': np.arange(147, 162),
        'other': 162
    },
    'Africa': {
        'main': np.arange(164, 173),
        'other': 173
    },
    'Asia': {
        'main': np.arange(175, 191),
        'other': 191
    },
    'Oceania': {
        'other': 192
    }
}

to_change = {
    'salvador': 'el salvador',
    'pays-bas': 'pays bas',
    'republique democratique du congo': 'congo rdc',
    'coree du sud': 'coree sud',
    'republique populaire de chine': 'chine',   
    'irak': 'iraq'
}

borough = 'Ville-Marie'
df = df[df['Arrondissement']==borough]


countries = df.columns[
    np.concatenate((indices['Americas']['main'], indices['Europe']['main'], indices['Africa']['main'], indices['Asia']['main']), axis=0)
    ].map(lambda s: unidecode(s).lower()).values

formatted_columns = df.columns.map(lambda s: unidecode(s).lower()).to_list()

print('Number of countries in the dataframe :', len(countries))
print(countries)

countries_found = []
color = []
for i, feature in enumerate(map_data['features']):
    
    # If country name is in the dataframe, use the listed value
    country_name = unidecode(feature['properties']['name_fr']).lower()
    if country_name in to_change.keys():
        country_name = to_change[country_name]
    print(country_name)
    if country_name in countries:
        idx = formatted_columns.index(country_name)
        color.append(df.iloc[0, idx])
        #print('|\t', country_name, color[-1])
        countries_found.append(country_name)
        
    # Elif the continent is in the dataframe, use the listed value for "Autres lieux de naissance..."
    elif feature['properties']['continent'] in ['South America', 'North America']:
        color.append(df.iloc[0, indices['Americas']['other']])
    elif feature['properties']['continent'] in ['Europe']:
        color.append(df.iloc[0, indices['Europe']['other']])
    elif feature['properties']['continent'] in ['Africa']:
        color.append(df.iloc[0, indices['Africa']['other']])
    elif feature['properties']['continent'] in ['Asia']:
        color.append(df.iloc[0, indices['Asia']['other']])
    elif feature['properties']['continent'] in ['Oceania']:
        color.append(df.iloc[0, indices['Oceania']['other']])

    # Else we don't plot the country (e.g. for Antarctica)
    else:
        color.append(None)

print(len(countries_found))
for country in countries:
    if country not in countries_found:
        print(country)
#print(len(color))
#print(color)


In [ ]:
fig = get_map(map_data, color)
fig.show()

# Test carte interactive

[Hugo] Bricolage d'une app de carte interactive en attendant d'avoir une app complète

In [ ]:
import os
import os.path as osp
import numpy as np
from dash import Dash, html, dcc, callback, Output, Input

from maps import *
from preprocess import *

PORT = 8000
ADDRESS = '127.0.0.1'

# Initialize the app - incorporate css
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(external_stylesheets=external_stylesheets)

# Load all the data
borough_df = get_boroughs_data()
montreal_mapdata = get_boroughs_mapdata()
world_mapdata = get_countries_mapdata()

# get the map of montreal
mtl_map_color = [borough_df.loc[borough_df['Arrondissement']==f['properties']['nom_arr'], 'Immigrante'].values[0] if f['properties']['nom_arr'] is not None else None for f in montreal_mapdata['features']]
montreal_map = get_map(montreal_mapdata, mtl_map_color, zoom='montreal', marker_line_width=1)
mtl_hovertemplate = "<b>Neighborhood :</b> %{customdata[0]}<br><b>Borough :</b> %{customdata[1]}<br><b>Immigrants :</b> %{z}<extra></extra>"
montreal_map.update_traces(
    showscale=False,
    hovertemplate=mtl_hovertemplate,
    customdata=[
        (
            f['properties']['nom_qr'],
            f['properties']['nom_arr'] if f['properties']['nom_arr'] is not None else '<i>No associated borough</i>',
        ) for f in montreal_mapdata['features']])
montreal_map.update_layout(
    map_style='white-bg',
    margin=dict(l=0, r=0, t=0, b=0),
    map=dict(center=dict(lat=45.5517, lon=-73.7073), zoom=9),
    width=400, height=600)

# App layout
app.layout = [
    # Column 1 : Settings
    html.Div(className='row', style={'margin-top':'30px'}, children=[

        # Column 1 : Montreal map
        html.Div(className='four columns', children=[
            dcc.Graph(id='montreal-map', figure=montreal_map, style={'justify': 'center'})]),

        # Column 2 : World map
        html.Div(className='eight columns', children=[
            dcc.Graph(id='world-map', style={'justify': 'center'})])])
]


# Plot the mesh corresponding to the histogram, via sampling if needed
@callback(
    Output(component_id='world-map', component_property='figure'),
    Input(component_id='montreal-map', component_property='clickData'))
def update_world_map(clickdata):
    if clickdata is None:
        color = get_countries_of_origin('Ville de Montréal', borough_df, world_mapdata)
        world_map = get_map(world_mapdata, color=color)
        world_map.update_layout(title='Countries of origin across Montreal boroughs')
    else:
        idx = clickdata['points'][0]['pointNumber']
        borough_name = montreal_mapdata['features'][idx]['properties']['nom_arr']

        # Some choropleth polygons actually have no associated borough
        if borough_name is None:
            world_map = get_map(world_mapdata, color=[None]*len(world_mapdata['features']))
        else:
            color = get_countries_of_origin(borough_name, borough_df, world_mapdata)
            world_map = get_map(world_mapdata, color)
            world_map.update_layout(title=f'Countries of origin for {borough_name}')
 
    world_map.update_layout(
        margin=dict(l=0, r=100, t=50, b=0),
        map=dict(center=dict(lat=50, lon=0), zoom=0.5),
        width=800, height=600)
    return world_map

# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=PORT, host=ADDRESS)